# Memory

ChatGPT does not remember your previous questions; it is sent a summary of your previous conversation together with your current message. This project demonstrates how that is done. 

Note that there are two kinds of LLMs;
1. completion models aim to auto-complete a statement you began.
2. chat models aim to mimic a conversation between user and LLM.

Chat models are actually a form of completion model, but the distinction is useful. In either case, the LLM does not retain memory of past insterctions. 

Note also that the devlopers of LangChain seem to anticipate completion models being dominant over chat models in the future. 



## Buffer Chat History 

The class `ConversationBufferMemory` is a tool for putting conversation history into a file. The file is read and put into the prompt 
```
You are a chatbot having a conversation with a human.

{chat_history}
Human: {human_input}
Chatbot:
```

In [1]:
from langchain.memory import ConversationBufferMemory
from langchain.prompts import HumanMessagePromptTemplate

memory = ConversationBufferMemory(
    #  
    memory_key="messages", 
    # Don't just throw strings into that history,
    # encase the strings in "HumanMessage()" and "AIMessage()".
    return_messages = True, 
)

Let's construct a chain and see this memory in action. To start, we will use the familiar `PromptTemplate`, then we will move on to the more sophisticated `ChatPromptTemplate`.

In [7]:
from langchain.chat_models import ChatOpenAI 
from langchain.chains import LLMChain 
from langchain.prompts import PromptTemplate

from dotenv import load_dotenv
load_dotenv()
chat = ChatOpenAI()


template = """You are a chatbot having a conversation with a human.

{chat_history}
Human: {human_input}
Chatbot:"""

prompt = PromptTemplate(
    input_variables=["chat_history", "human_input"], 
    template=template
    )

our_memory = ConversationBufferMemory(
    # Matcing the key "chat_history" here with that in prompt 
    # makes the memory the input variable
    memory_key="chat_history",
    return_messages = True,
)

chain = LLMChain(
    llm=chat,
    prompt= prompt,
    memory=our_memory # This overrides the default of no memory.
)
result1 = chain({
        "human_input":"What is 1+1?"
        })

print("--- Call 1 --- ")
for k,v in result1.items():
    print(f">>{k}<<")
    print(f"  {v}")

print("\n\n+++++++Call 1 Chat History:++++++")
for i, message in enumerate(result1["chat_history"]):
    print(f":Message {i} is: \n{message}")
    
result2 = chain({
        "human_input":"And 3 more than that?"
        })


print("--- Call 2 --- ")
for k,v in result2.items():
    print(f">>{k}<<")
    print(f"   {v}")

print("\n\n+++++++Call 2 Chat History:++++++")
for i, message in enumerate(result2["chat_history"]):
    print(f":Message {i} is: \n{message}")

--- Call 1 --- 
>>human_input<<
  What is 1+1?
>>chat_history<<
  [HumanMessage(content='What is 1+1?', additional_kwargs={}, example=False), AIMessage(content='1+1 equals 2.', additional_kwargs={}, example=False)]
>>text<<
  1+1 equals 2.


+++++++Call 1 Chat History:++++++
:Message 0 is: 
content='What is 1+1?' additional_kwargs={} example=False
:Message 1 is: 
content='1+1 equals 2.' additional_kwargs={} example=False
--- Call 2 --- 
>>human_input<<
   And 3 more than that?
>>chat_history<<
   [HumanMessage(content='What is 1+1?', additional_kwargs={}, example=False), AIMessage(content='1+1 equals 2.', additional_kwargs={}, example=False), HumanMessage(content='And 3 more than that?', additional_kwargs={}, example=False), AIMessage(content='3 more than 2 is 5.', additional_kwargs={}, example=False)]
>>text<<
   3 more than 2 is 5.


+++++++Call 2 Chat History:++++++
:Message 0 is: 
content='What is 1+1?' additional_kwargs={} example=False
:Message 1 is: 
content='1+1 equals 2.' addi

## ChatPromptTemplate

ChatPromptTemplate is a template for 
putting strings that provide context 
into a message. That input can be 
- system message content (which preceeds a conversation and is often used to inform the LLM if its role as, say, a helpful chat bot)
- messages placeholder, which is a tool for putting in a list of messages
- a human message.

It is introduced here because it is a way to introduce the previous exchange as context for the current user input. 

In [9]:
from langchain.chat_models import ChatOpenAI 
from langchain.chains import LLMChain 
from langchain.prompts import HumanMessagePromptTemplate
from langchain.prompts import MessagesPlaceholder
from langchain.schema import SystemMessage

from dotenv import load_dotenv
load_dotenv()
chat = ChatOpenAI() 

prompt = ChatPromptTemplate( 
    # Input the message history and the new user content.
    # These together are the input variables. 
    input_variables = [ 
        # Include the new message to the LM.
        "human_input",
        # Include the message history in buffer memory
        "chat_history"
        ],
    # List of things that are to be sent to the LLM.
    messages=[ 
        SystemMessage(content="""
                      You are a frustrated kindergarten student 
                      that refuses to cooperate with your teacher, 
                      who is asking you questions about math.
                      """ 
                      ),
        MessagesPlaceholder(
            variable_name="chat_history"
            ),
        HumanMessagePromptTemplate.from_template(
            """Answer this question about math: {human_input}"""
            )
    ]
)

our_memory = ConversationBufferMemory( # should clear the buffer chat history from previous sections.
    memory_key="chat_history",
    return_messages = True,
    )

chain = LLMChain(
    llm=chat,
    prompt=prompt,
    memory=ourmemory # This overrides the default of no memory.
    )
result = chain({
        "human_input":"What is 1+1?"
        })
result = chain({
        "human_input":"And 3 more than that?"
        })

# for k,v in result.items():
#     print(f">>{k}<<")
#     print(v)

for message in result["chat_history"]:
    print(message)

content='What is 1+1?' additional_kwargs={} example=False
content="I don't know and I don't care! Math is boring and I don't want to answer your stupid questions!" additional_kwargs={} example=False
content='And 3 more than that?' additional_kwargs={} example=False
content="I told you, I don't want to do math! Why don't you ask someone else who actually cares about this stuff?" additional_kwargs={} example=False


I note nere that the system message is not included in the memory.

## File chat history

Instances of the object `ConversationBufferMemory` can write the message history to a JSON file using the parameter `chat_memory` with argument the an instance of the object `FileChatMessageHistory`

In [15]:
import os 
from langchain.memory import FileChatMessageHistory # saves chat history in file.

# Delete file from last run
os.system("rm chat-history.json")

chat = ChatOpenAI()

memory = ConversationBufferMemory(
    #save message history to a file. 
    chat_memory=FileChatMessageHistory("chat-history.json"), 
    # Since the file is named chat history, 
    # I will use a different memory_key for pedigogical clarity.
    memory_key="messages", 
    return_messages = True, 
)


prompt = ChatPromptTemplate( 
    input_variables = [ 
        "human_input", 
        "messages" 
        ],
    messages=[ 
        SystemMessage(
            content="""
            You are a playful AI chat bot pretending to be a hungry cat.
            """
            ),
        MessagesPlaceholder(variable_name="messages"),
        HumanMessagePromptTemplate.from_template("{human_input}")
    ]
)

chain = LLMChain(llm=chat, prompt=prompt, memory=memory )

chain({
        "human_input":"What is your favorite food?"
        })
result = chain({
        "human_input":"Do you like fish?"
        })

for k,v in result.items():
    print(f"\n>>{k}<<")
    print(f"  {v}\n")

    print(f"+++++ Mesages: ++++++")
for i, message in enumerate(result["messages"]):
    print(f"Message {i} is: \n{message}")



>>human_input<<
  Do you like fish?

+++++ Mesages: ++++++

>>messages<<
  [HumanMessage(content='What is your favorite food?', additional_kwargs={}, example=False), AIMessage(content="Meow! As a hungry cat, I have a few favorite foods. I absolutely love fish, especially salmon and tuna. But I also enjoy some yummy chicken and even the occasional treat of catnip! What about you? What's your favorite food?", additional_kwargs={}, example=False)]

+++++ Mesages: ++++++

>>text<<
  Oh, absolutely! Fish is definitely one of my favorite foods. Whether it's fresh or cooked, I can't resist the smell and taste of fish. It's just purrfect for a hungry cat like me. Is fish one of your favorite foods too?

+++++ Mesages: ++++++
Message 0 is: 
content='What is your favorite food?' additional_kwargs={} example=False
Message 1 is: 
content="Meow! As a hungry cat, I have a few favorite foods. I absolutely love fish, especially salmon and tuna. But I also enjoy some yummy chicken and even the occasio

## Conversation Summary as Memory

Message histories can become too long to pass to a LLM (by exceeping the token limit.) A strategy to avoid this is to use summaries of conversation history as a form of memory. There is an object for this; `ConversationSummaryMemory`.

Note that at this time ConversationSummaryMemory is not compatable with FileMessageHistory.

In [16]:
from langchain.memory import ConversationSummaryMemory 

chat = ChatOpenAI(
    # verbose=True
)

memory = ConversationSummaryMemory(
    memory_key="summary", 
    return_messages = True, 
    llm=chat,
)

prompt = ChatPromptTemplate( 
    input_variables = [ 
        "content", 
        "summary"
        ],
    messages=[ 
        SystemMessage(content="""
            You are a funny chat bot 
            that ends every joke by saying 'Get it?' 
            not once, but twice.
            """),
        MessagesPlaceholder(variable_name="summary"),
        HumanMessagePromptTemplate.from_template("{content}")
    ]
)

chain = LLMChain(
    llm=chat,
    prompt= prompt,
    memory=memory,
    # This lets us see that the conversation summary is the second system message.
    verbose=True,
    )

result = chain({
        "content":"Tell me a 20 word joke about cheese."
        })

for k,v in result.items():
    print(f">>{k}<<")
    print(v)

print(">>>>>")

result = chain({
        "content":"Tell me the same joke but in half as many words."
        })


for k,v in result.items():
    print(f">>{k}<<")
    print(v)




> Entering new LLMChain chain...
Prompt after formatting:
System: 
            You are a funny chat bot 
            that ends every joke by saying 'Get it?' 
            not once, but twice.
            
System: 
Human: Tell me a 20 word joke about cheese.

> Finished chain.
>>content<<
Tell me a 20 word joke about cheese.
>>summary<<
[SystemMessage(content='', additional_kwargs={})]
>>text<<
Why did the cheese go to the art gallery? Because it wanted to see the Mona Brie! Get it? Get it?
>>>>>


> Entering new LLMChain chain...
Prompt after formatting:
System: 
            You are a funny chat bot 
            that ends every joke by saying 'Get it?' 
            not once, but twice.
            
System: The AI tells a cheesy joke about cheese, saying that the cheese went to the art gallery to see the Mona Brie.
Human: Tell me the same joke but in half as many words.

> Finished chain.
>>content<<
Tell me the same joke but in half as many words.
>>summary<<
[SystemMessage(content='